# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [3]:
import pandas as pd
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_dt = (pd.read_csv('../../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))


# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [4]:
# create X and Y
X = adult_dt.drop('income', axis=1)  # All columns except 'income'
Y = adult_dt['income']               # Just the 'income' column

# split the data using train_test_split
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(
    X,                  # Features
    Y,                  # Target
    test_size=0.3,      # 30% for testing, 70% for training
    random_state=42     # For reproducibility
)

# verify 
print("Dataset shapes:")
print(f"Original data: {adult_dt.shape}")
print("\nFeatures (X):")
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print("\nTarget (Y):")
print(f"Y_train shape: {Y_train.shape}")
print(f"Y_test shape: {Y_test.shape}")

# Verify the split proportions
print("\nSplit proportions:")
print(f"Training set: {len(X_train) / len(X):.2%}")
print(f"Testing set: {len(X_test) / len(X):.2%}")

Dataset shapes:
Original data: (32561, 15)

Features (X):
X_train shape: (22792, 14)
X_test shape: (9769, 14)

Target (Y):
Y_train shape: (22792,)
Y_test shape: (9769,)

Split proportions:
Training set: 70.00%
Testing set: 30.00%


## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?

The random state of the splitting function is set to 42.   Since setting this fixed value ensures that every time the code is run, it will produce the exact same train-test split of the data allowing others to reproduce the same results.


# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.pipeline import Pipeline  # Added this import

# identify numerical and categorical columns
numerical_features = ['age', 'fnlwgt', 'education-num', 'capital-gain', 
                     'capital-loss', 'hours-per-week']

categorical_features = ['workclass', 'education', 'marital-status', 'occupation',
                       'relationship', 'race', 'sex', 'native-country']

# Create the numeric pipeline
numeric_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=7, weights='distance')),
    ('scaler', RobustScaler())
])

# Create the categorical pipeline
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', drop='binary'))
])

# Create the column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# verify the structure
from pprint import pprint
print("Column Transformer Structure:")
pprint(preprocessor.get_params())

Column Transformer Structure:
{'cat': Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                ('onehot',
                 OneHotEncoder(drop='binary', handle_unknown='ignore'))]),
 'cat__imputer': SimpleImputer(strategy='most_frequent'),
 'cat__imputer__add_indicator': False,
 'cat__imputer__copy': True,
 'cat__imputer__fill_value': None,
 'cat__imputer__keep_empty_features': False,
 'cat__imputer__missing_values': nan,
 'cat__imputer__strategy': 'most_frequent',
 'cat__memory': None,
 'cat__onehot': OneHotEncoder(drop='binary', handle_unknown='ignore'),
 'cat__onehot__categories': 'auto',
 'cat__onehot__drop': 'binary',
 'cat__onehot__dtype': <class 'numpy.float64'>,
 'cat__onehot__feature_name_combiner': 'concat',
 'cat__onehot__handle_unknown': 'ignore',
 'cat__onehot__max_categories': None,
 'cat__onehot__min_frequency': None,
 'cat__onehot__sparse_output': True,
 'cat__steps': [('imputer', SimpleImputer(strategy='most_frequent')),
                ('on

## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [11]:

from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

#full pipeline
pipe = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('classifier', RandomForestClassifier())
])

# pipeline structure
print("Pipeline steps:")
for step_name, step in pipe.named_steps.items():
    print(f"\n{step_name}:")
    print(step)


Pipeline steps:

preprocessing:
ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  KNNImputer(n_neighbors=7,
                                                             weights='distance')),
                                                 ('scaler', RobustScaler())]),
                                 ['age', 'fnlwgt', 'education-num',
                                  'capital-gain', 'capital-loss',
                                  'hours-per-week']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEncoder(drop='binary',
                                                                handle_unknown='ignore'))]),
                

# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [16]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, log_loss, roc_auc_score
from sklearn.metrics import accuracy_score, balanced_accuracy_score

# identify numerical and categorical columns
numerical_features = ['age', 'fnlwgt', 'education-num', 'capital-gain', 
                     'capital-loss', 'hours-per-week']

categorical_features = ['workclass', 'education', 'marital-status', 'occupation',
                       'relationship', 'race', 'sex', 'native-country']

# Create the numeric pipeline
numeric_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=7, weights='distance')),
    ('scaler', RobustScaler())
])

# Create the categorical pipeline (fixed the drop parameter)
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', drop='if_binary'))
])

# Create the column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Create the full pipeline
pipe = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('classifier', RandomForestClassifier())
])

# Define scoring metrics
scoring = {
    'neg_log_loss': 'neg_log_loss',
    'roc_auc': 'roc_auc',
    'accuracy': 'accuracy',
    'balanced_accuracy': 'balanced_accuracy'
}

# Perform cross-validation
cv_results = cross_validate(
    pipe,
    X_train,
    Y_train,
    scoring=scoring,
    cv=5,                # 5 folds
    return_train_score=True  # To get both training and validation scores
)

# Convert results to a DF and sort by test neg_log_loss
cv_df = pd.DataFrame({
    'train_neg_log_loss': cv_results['train_neg_log_loss'],
    'test_neg_log_loss': cv_results['test_neg_log_loss'],
    'train_roc_auc': cv_results['train_roc_auc'],
    'test_roc_auc': cv_results['test_roc_auc'],
    'train_accuracy': cv_results['train_accuracy'],
    'test_accuracy': cv_results['test_accuracy'],
    'train_balanced_accuracy': cv_results['train_balanced_accuracy'],
    'test_balanced_accuracy': cv_results['test_balanced_accuracy']
})



/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [17]:
# Display fold-level results sorted by test neg_log_loss
print("Fold-level results sorted by test neg_log_loss:")
print(cv_df.sort_values('test_neg_log_loss', ascending=False))



Fold-level results sorted by test neg_log_loss:
   train_neg_log_loss  test_neg_log_loss  train_roc_auc  test_roc_auc  \
0           -0.081373          -0.347234            1.0      0.906596   
1           -0.081234          -0.369813            1.0      0.900860   
3           -0.081596          -0.377562            1.0      0.906721   
2           -0.081754          -0.388913            1.0      0.901826   
4           -0.081203          -0.402065            1.0      0.901364   

   train_accuracy  test_accuracy  train_balanced_accuracy  \
0        0.999945       0.851722                 0.999887   
1        1.000000       0.850844                 1.000000   
3        0.999945       0.857394                 0.999887   
2        1.000000       0.851251                 1.000000   
4        1.000000       0.856077                 1.000000   

   test_balanced_accuracy  
0                0.777359  
1                0.770634  
3                0.781573  
2                0.772596  
4     

Calculate the mean of each metric. 

In [18]:
# Calculate mean of each metric
print("\nMean of each metric:")
means = cv_df.mean()
print(means)




Mean of each metric:
train_neg_log_loss        -0.081432
test_neg_log_loss         -0.377117
train_roc_auc              1.000000
test_roc_auc               0.903473
train_accuracy             0.999978
test_accuracy              0.853458
train_balanced_accuracy    0.999955
test_balanced_accuracy     0.776020
dtype: float64


Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [19]:
# Calculate metrics on test set
pipe.fit(X_train, Y_train)
Y_pred_proba = pipe.predict_proba(X_test)
Y_pred = pipe.predict(X_test)

# Calculate test metrics
test_results = {
    'neg_log_loss': -log_loss(Y_test, Y_pred_proba),
    'roc_auc': roc_auc_score(Y_test, Y_pred_proba[:, 1]),
    'accuracy': accuracy_score(Y_test, Y_pred),
    'balanced_accuracy': balanced_accuracy_score(Y_test, Y_pred)
}

print("\nTest set metrics:")
print(test_results)


Test set metrics:
{'neg_log_loss': -0.3682265163220453, 'roc_auc': 0.9003652569406646, 'accuracy': 0.8552564233800799, 'balanced_accuracy': 0.7743361928992567}


# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

(Answer here.)

Using numeric values instead of strings is more efficient for memory usage and computations.

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.